In [33]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem.porter import PorterStemmer
import re
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
! unzip '/content/drive/MyDrive/IMDB Dataset.csv (1).zip' -d '/content/drive/MyDrive/IMDB Dataset.csv (1)'

Archive:  /content/drive/MyDrive/IMDB Dataset.csv (1).zip
replace /content/drive/MyDrive/IMDB Dataset.csv (1)/IMDB Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/drive/MyDrive/IMDB Dataset.csv (1)/IMDB Dataset.csv  


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
df=pd.read_csv('/content/drive/MyDrive/IMDB Dataset.csv (1)/IMDB Dataset.csv', nrows=1000)

In [6]:
df.shape

(1000, 2)

In [7]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [8]:
import re
def remove_html_tags(text):
  pattern=re.compile('<.*?>')
  return pattern.sub(r'',text)

In [9]:
df['review']=df['review'].apply(remove_html_tags)
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
995,Nothing is sacred. Just ask Ernie Fosselius. T...,positive
996,I hated it. I hate self-aware pretentious inan...,negative
997,I usually try to be professional and construct...,negative
998,If you like me is going to see this in a film ...,negative


In [10]:
def remove_url(text):
  pattern=re.compile(r'https?://\S+|www\.\S+')
  return pattern.sub(r'',text)

In [11]:
df['review']=df['review'].apply(remove_url)
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
995,Nothing is sacred. Just ask Ernie Fosselius. T...,positive
996,I hated it. I hate self-aware pretentious inan...,negative
997,I usually try to be professional and construct...,negative
998,If you like me is going to see this in a film ...,negative


In [12]:
import string,time
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [13]:
def remove_punc(text):
  pattern = re.compile(f'[{re.escape(string.punctuation)}]')
  return pattern.sub(r'', text)

In [14]:
df['review']=df['review'].apply(remove_punc)
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production The filming tech...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically theres a family where a little boy J...,negative
4,Petter Matteis Love in the Time of Money is a ...,positive
...,...,...
995,Nothing is sacred Just ask Ernie Fosselius The...,positive
996,I hated it I hate selfaware pretentious inanit...,negative
997,I usually try to be professional and construct...,negative
998,If you like me is going to see this in a film ...,negative


In [15]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
stopwords.words('english')

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 'her',
 'here',
 'hers',
 'herself',
 "he's",
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 'if',
 "i'll",
 "i'm",
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 "i've",
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [17]:
def remove_stopwords(text):
    new_text = []

    for word in text.split():
        if word in stopwords.words('english'):
            new_text.append('')
        else:
            new_text.append(word)
    x = new_text[:]
    new_text.clear()
    return " ".join(x)

In [18]:
df['review']=df['review'].apply(remove_stopwords)
df

,review,sentiment
0,One reviewers mentioned watching 1 Oz e...,positive
1,A wonderful little production The filming tech...,positive
2,I thought wonderful way spend time hot ...,positive
3,Basically theres family little boy Jake thi...,negative
4,Petter Matteis Love Time Money visually s...,positive
...,...,...
995,Nothing sacred Just ask Ernie Fosselius These...,positive
996,I hated I hate selfaware pretentious inanity ...,negative
997,I usually try professional constructive I ...,negative
998,If like going see film history class s...,negative


In [20]:
vocab_size = 10000  # Adjust based on your dataset
max_length = 100    # Adjust based on your dataset

# Initialize tokenizer
tokenizer = Tokenizer(num_words=vocab_size, oov_token='')
tokenizer.fit_on_texts(df['review'])

# Convert texts to sequences
sequences = tokenizer.texts_to_sequences(df['review'])

# Pad sequences
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')


In [25]:
vocab_size = 10000  # Example vocab size
max_length = 100    # Example max sequence length

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=64, input_shape=(max_length,)),
    GRU(64, return_sequences=False),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 100, 64)        │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_3 (GRU)                     │ (None, 64)             │        24,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 665,025 (2.54 MB)

 Trainable params: 665,025 (2.54 MB)

 Non-trainable params: 0 (0.00 B)

In [43]:
from sklearn.preprocessing import LabelEncoder
# Assume df has 'review' (text) and 'sentiment' (labels)
texts = df['review'].values
labels = df['sentiment'].values

# Encode labels if they are strings
if labels.dtype == 'O':
    labels = LabelEncoder().fit_transform(labels)
labels = labels.astype('float32')

# Tokenize and pad
vocab_size = 10000
max_length = 100
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
X = pad_sequences(sequences, maxlen=max_length, padding='post')

# Build model
model = Sequential([
    Embedding(vocab_size, 64, input_shape=(max_length,)),
    GRU(64),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
history = model.fit(X, labels, epochs=8, validation_split=0.02, verbose=1)


Epoch 1/8


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


31/31 ━━━━━━━━━━━━━━━━━━━━ 8s 99ms/step - accuracy: 0.4783 - loss: 0.6944 - val_accuracy: 0.4500 - val_loss: 0.6925
Epoch 2/8
31/31 ━━━━━━━━━━━━━━━━━━━━ 4s 77ms/step - accuracy: 0.6170 - loss: 0.6880 - val_accuracy: 0.4500 - val_loss: 0.6891
Epoch 3/8
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 97ms/step - accuracy: 0.5994 - loss: 0.6695 - val_accuracy: 0.5500 - val_loss: 0.6540
Epoch 4/8
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 100ms/step - accuracy: 0.6950 - loss: 0.5883 - val_accuracy: 0.7000 - val_loss: 0.5792
Epoch 5/8
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 78ms/step - accuracy: 0.6882 - loss: 0.5456 - val_accuracy: 0.7000 - val_loss: 0.5414
Epoch 6/8
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 78ms/step - accuracy: 0.7361 - loss: 0.4521 - val_accuracy: 0.7500 - val_loss: 0.4887
Epoch 7/8
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - accuracy: 0.7703 - loss: 0.4012 - val_accuracy: 0.8000 - val_loss: 0.4437
Epoch 8/8
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 91ms/step - accuracy: 0.7938 - loss: 0.3514 - val_accuracy: 0.8000 - val_loss: 0.4209


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/ColabNotebooks/gru_model.keras'